In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,inspect

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
import csv,sqlite3


In [2]:
con = sqlite3.connect("./db/flights_data.sqlite")


In [3]:
df = pd.read_csv("./db/flights_metadata.csv")
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0,NaN


In [4]:
del df['Unnamed: 21']

In [5]:
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,0.0,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0


In [6]:
df = df.rename(columns = {" month":"month"," weather_ct":"weather_ct"," arr_delay":"arr_delay"," carrier_delay":"carrier_delay"})

In [7]:
df = df[["year","month","carrier","carrier_name","airport","airport_name","arr_flights","arr_del15","carrier_ct","weather_ct",
"nas_ct","security_ct","late_aircraft_ct","arr_cancelled","arr_diverted","arr_delay","carrier_delay",
"weather_delay","nas_delay","security_delay","late_aircraft_delay"]]

In [8]:
df.head(1)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.8,...,0.0,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0


In [9]:
df.to_sql("flights_data", con =con, if_exists='replace', index=False)


In [10]:
df.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
dtype: object

In [11]:
engine = create_engine("sqlite:///db/flights_data.sqlite", encoding='utf8')
conn = engine.connect()

In [12]:
inspector = inspect(engine)
inspector.get_table_names()
inspector.get_columns("flights_data")


[{'name': 'year',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'month',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'carrier',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'carrier_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'airport',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'airport_name',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'arr_flights',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'arr_del15',
  'type': REAL(),
  'nullable': True,
  'default': None,
  'autoincrement': 'aut

In [13]:
session = Session(engine)

In [14]:
flight_data = pd.read_sql("SELECT * FROM flights_data",engine)
flight_data.head(2)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2013,1,9E,Pinnacle Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",139.0,37.0,13.04,0.80,...,0.00,9.24,2.0,5.0,1860.0,789.0,75.0,451.0,0.0,545.0
1,2013,1,9E,Pinnacle Airlines Inc.,JFK,"New York, NY: John F. Kennedy International",1421.0,285.0,69.88,10.89,...,1.18,65.72,67.0,1.0,22046.0,5810.0,1495.0,8501.0,72.0,6168.0


In [15]:
flight_data.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
dtype: object